In [2]:
from bs4 import BeautifulSoup 
import urllib.request
import ssl
from dotenv import load_dotenv
import openai
import os
import requests
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import pandas as pd
import json
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re
from sqlalchemy import inspect
import sqlalchemy
from sqlalchemy import text 
import chromadb

In [41]:
ssl._create_default_https_context = ssl._create_stdlib_context
html_link = "https://pages.stern.nyu.edu/~adamodar/New_Home_Page/datacurrent.html"

with urllib.request.urlopen(html_link) as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    soup = BeautifulSoup(s,"lxml")

html_table = soup.find_all("table")
req_table = html_table[1]
hrefs_list = req_table.find_all('a')

In [42]:
req_href = {"US":[],"Europe":[],"Japan":[],"AUS_NZ_CANADA":[],"Emerging":[],"China":[],"India":[],"Global":[]}

for i in hrefs_list:
    name = i.get_text().strip()
    try:
        href_attr = i['href']
        if href_attr.endswith('.xls'):
            if "US" in name:
                req_href["US"].append(href_attr)
            elif "Europe" in name:
                req_href["Europe"].append(href_attr)
            elif "Japan" in name:
                req_href["Japan"].append(href_attr)
            elif "Aus" in name:
                req_href['AUS_NZ_CANADA'].append(href_attr)
            elif "Emerging" in name:
                req_href['Emerging'].append(href_attr)
            elif "China" in name:
                req_href['China'].append(href_attr)
            elif "India" in name:
                req_href['India'].append(href_attr)
            elif "Global" in name: 
                req_href['Global'].append(href_attr)
    except:
        pass

In [93]:

ssl._create_default_https_context = ssl._create_stdlib_context

import os
os.makedirs("DATA",exist_ok=True)
for country,excel_files in req_href.items():
    country_path = os.path.join("DATA",country) 
    os.makedirs(country_path,exist_ok=True)
    for file in excel_files:
        file_name = file.split("/")[-1].split(".")[0]
        full_file_name = os.path.join(country_path,f"{file_name}.xls")
        resp = requests.get(file,verify=False)
        output = open(full_file_name, 'wb')
        output.write(resp.content)
        output.close()

In [35]:
for country in os.listdir("DATA"):
    dir_len = len(os.listdir(os.path.join("DATA",country)))
    country_len = len(req_href[country])
    print(f'FOR {country} WE HAVE DIRECTORY LEN = {dir_len} and ACTUAL LEN = {country_len}')

FOR Emerging WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29
FOR Europe WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29
FOR Global WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29
FOR AUS_NZ_CANADA WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29
FOR China WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29
FOR Japan WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29
FOR US WE HAVE DIRECTORY LEN = 24 and ACTUAL LEN = 24
FOR India WE HAVE DIRECTORY LEN = 29 and ACTUAL LEN = 29


In [38]:
import pandas as pd
from tqdm import tqdm
import json
pd.set_option('display.max_rows', 50)

def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)

dir = "DATA2"
processed_dir = "Processed Data"
all_infos_dict = []
os.makedirs(processed_dir,exist_ok=True)
for country in os.listdir(dir):
    print(country)
    file_name = os.path.join(dir,country)
    os.makedirs(os.path.join(processed_dir,country),exist_ok=True)
    os.makedirs(file_name,exist_ok=True)
    for excel_file in tqdm(os.listdir(file_name)):
        full_file_name = os.path.join(file_name,excel_file)
        xls = pd.ExcelFile(full_file_name)
        sns = xls.sheet_names
        for sheet_name in sns:
            if "Var" in sheet_name or "var" in sheet_name:
                info_df = xls.parse(sheet_name)
                info_df = xls.parse(sheet_name)
                info_df.dropna(how="all",inplace=True)
                info_dict = {}
                for cols in info_df.columns:
                    if "End" not in cols and 'Unnamed' not in cols:
                        info_dict['Summary'] = cols
                info_dict['Vars'] = info_df.values[1:].tolist()
                all_infos_dict.append(info_dict)
            elif "Industry" in sheet_name or "industry" in sheet_name:
                data_df = xls.parse(sheet_name)
        try:
            data_df.dropna(axis=1,thresh=5,inplace=True)
            data_df.dropna(inplace=True)
            new_header = data_df.iloc[0] #grab the first row for the header
        except:
            print(full_file_name)
            print(data_df)
        data_df = data_df[1:] #take the data less the header row
        data_df.reset_index(inplace=True,drop=True)
        new_header = [sanitize_column_name(str(col)) for col in new_header]
        data_df.columns = new_header #set the header row as the df header
        save_name = full_file_name.split(".")[0].split("/")[-1]
        save_file_path = os.path.join(os.path.join(processed_dir,country),save_name)
        data_df.to_csv(save_file_path+".csv",index=False)
        # with open(save_file_path+".json", "w") as outfile: 
        #     json.dump(info_dict, outfile)


Emerging


100%|██████████| 29/29 [00:00<00:00, 86.52it/s]


Europe


100%|██████████| 29/29 [00:00<00:00, 94.75it/s]


Global


...

100%|██████████| 29/29 [00:00<00:00, 66.34it/s] 


US


100%|██████████| 24/24 [00:00<00:00, 102.91it/s]


India


100%|██████████| 29/29 [00:00<00:00, 100.35it/s]


In [196]:
all_infos_dict[0]

{'Summary': 'Measures of accounting returns, to all claim holders (and from operations)',
 'Vars': [['Number of firms',
   'Number of firms in the industry grouping.',
   'Law of large numbers?'],
  ['R&D Capitalized',
   'My estimate of R&D capitalization, based upon a 5-year straight line amortization period, aggregated across firms in the group',
   'Capitalized value of R&D gets added on to book equity and to invested capital'],
  ['Capitalized R&D as percent of invested capital',
   'My R&D capitalization estimate, as a percent of invested capital including that number, based upon aggregated values across firms',
   'Magnitude of investment in R&D, relative to investment in more traditional capital expenditures.'],
  ['R&D -LTM',
   'Aggregated R&D expenses across the last twelve months, across companies in the group.',
   'Spending on R&D in most recent year'],
  ['R&D: Years minus 1 to minus 5',
   'Aggregated R&D expenses for each of the previous five years, across companies in

In [197]:
df = pd.read_csv("Processed Data/US/capex.csv")
df.head()

,Industry_Name,Number_of_Firms,Capital_Expenditures_(US_$_millions),Depreciation_&_Amort_((US_$_millions),Cap_Ex/Deprecn,Acquisitions_(US_$_millions),Net_R&D_(US_$_millions),Net_Cap_Ex/Sales,Net_Cap_Ex/_EBIT_(1-t),Sales/_Invested_Capital_(LTM)
0,Advertising,57,775.729,1887.338,0.411018,322.559,75.0800,-0.016886,-0.218120,3.283403
1,Aerospace/Defense,70,10982.128,13311.598,0.825004,10344.750,830.4634,0.022829,0.318077,1.984340
2,Air Transport,25,25559.725,10609.948,2.409034,368.210,73.5486,0.067203,1.355173,1.777320
3,Apparel,38,1730.981,1386.480,1.248472,38.739,0.9474,0.005365,0.072557,1.773076
4,Auto & Truck,34,29899.486,18677.668,1.600815,193.220,983.0696,0.026577,0.675918,1.048732


## BUILD TABLE NAMES AND METADATA

In [199]:
df.head(10).to_csv()

',Industry_Name,Number_of_Firms,Capital_Expenditures_(US_$_millions),Depreciation_&_Amort_((US_$_millions),Cap_Ex/Deprecn,Acquisitions_(US_$_millions),Net_R&D_(US_$_millions),Net_Cap_Ex/Sales,Net_Cap_Ex/_EBIT_(1-t),Sales/_Invested_Capital_(LTM)\n0,Advertising,57,775.729,1887.338,0.4110175283918408,322.559,75.07999999999993,-0.0168860862168837,-0.2181203933865563,3.283403041333076\n1,Aerospace/Defense,70,10982.128,13311.597999999998,0.8250044810547916,10344.749999999998,830.4633999999933,0.0228292321038578,0.318077061754813,1.9843395804666923\n2,Air Transport,25,25559.725,10609.948,2.4090339556800844,368.21,73.5486000000019,0.0672027546276203,1.3551729988499104,1.7773199625336142\n3,Apparel,38,1730.9809999999998,1386.4799999999996,1.2484716692631703,38.739,0.9473999999991064,0.0053650838666078,0.0725567415076527,1.7730762352050575\n4,Auto & Truck,34,29899.486,18677.668,1.6008147269776931,193.22,983.0695999999988,0.0265766732899557,0.6759175844836917,1.048731678622114\n5,Auto Parts,39,35

In [6]:

load_dotenv(override=True)
openai.api_key = os.environ['OPENAI_API_KEY']

In [8]:
import dspy
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
dspy.settings.configure(lm=turbo)

class SQLTableMetadata(dspy.Signature):
    """Give a suitable table name and description about the given table"""
    pandas_dataframe_str = dspy.InputField(desc="First 10 rows of a pandas dataframe delimited by newline character")
    table_name = dspy.OutputField(desc="suitable table name")
    table_summary = dspy.OutputField(desc="a summary about the table")

class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(SQLTableMetadata)
    
    def forward(self, pandas_dataframe_str):
        return self.prog(pandas_dataframe_str=pandas_dataframe_str)

cot = CoT()

# cot(pandas_dataframe_str = df.head(10).to_csv())


In [9]:
processed_dir = "Processed Data"

for country in os.listdir(processed_dir):
    country_folder = os.path.join(processed_dir,country)
    print(f"{country}")
    for files in tqdm(os.listdir(country_folder)):
        # print(files)
        if files.endswith(".csv"):
            file_name = files.split(".")[0]
            csv_file_path = os.path.join(country_folder,files)
            df = pd.read_csv(csv_file_path,index_col=False)
            json_file_path = os.path.join(country_folder,f"{file_name}.json")
            with open(json_file_path,'r') as f:
                data = json.loads(f.read())
            if 'table_name' in data and 'table_summary' in data:
                # if data['table_name'] == "" or data['table_summary'] == "":
                if data['table_summary'] == "":
                    pass
                else:
                    continue
            table_preds = cot(pandas_dataframe_str = df.head(10).to_csv())
            data['table_name'] = table_preds.table_name
            data['table_summary'] = table_preds.table_summary
            with open(json_file_path,'w') as f:
                json.dump(data, f)

US


100%|██████████| 48/48 [08:18<00:00, 10.39s/it]


India


100%|██████████| 58/58 [12:00<00:00, 12.43s/it]


## NEXT TASKS
1. Build database with each region for each table
2. Embed the table summary and table SCHEMA. Also, embed the table rows
3. Retrieval at table level and embed the rows to retrieve relevant rows from the retrieved schema of table
4. Final LLM call

In [3]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()

## DATABASE FOR US

In [4]:
processed_dir  = "Processed Data"
def sqlalchemy_engine(region:str):
    """Create a SQLAlchemy engine for the given region"""
    assert region in os.listdir(processed_dir), f"{region} is not a valid region from {os.listdir(processed_dir)}"
    engine = create_engine(f"sqlite:///{region}.db")
    metadata_obj = MetaData()
    region_path = os.path.join(processed_dir,region)
    dfs = []
    for dataframes_path in os.listdir(region_path):
        if dataframes_path.endswith(".csv"):
            df = pd.read_csv(os.path.join(region_path,dataframes_path),index_col=False)
            dfs.append((dataframes_path,df))
    pbar = tqdm(total=len(dfs),desc=f"Creating tables for {region}")
    for _, df_table_name in enumerate(dfs):
        table_name = df_table_name[0]
        table_name = table_name.split(".")[0]
        df = df_table_name[1]
        # print(f"Creating table: {table_name}")
        create_table_from_dataframe(df,table_name, engine, metadata_obj)
        # print(f"Done creating table for: {table_name}")
        pbar.update(1)
    return engine

In [5]:
us_engine = sqlalchemy_engine("US")
india_engine = sqlalchemy_engine("India")
china_engine = sqlalchemy_engine("China")
europe_engine = sqlalchemy_engine("Europe")
global_engine = sqlalchemy_engine("Global")
aus_nz_canada_engine = sqlalchemy_engine("AUS_NZ_CANADA")
japan_engine = sqlalchemy_engine("Japan")
emerging_engine = sqlalchemy_engine("Emerging")

Creating tables for Emerging: 100%|██████████| 29/29 [00:02<00:00, 11.46it/s]


In [6]:
def get_table_infos(sql_engine:sqlalchemy.engine.base.Engine,region:str):
    """Get all tables in the database"""
    inspector = inspect(sql_engine)
    table_names = inspector.get_table_names()
    table_infos_dict = {tb: [] for tb in table_names}
    for tb in table_names:
        column_dict = inspector.get_columns(tb)
        schema_str = ""
        primary_keys = []
        for col in column_dict:
            schema_str += f"{col['name']} ({col['type']}), "
            if col["primary_key"] not in primary_keys:
                primary_keys.append(col["name"])
        with open(os.path.join(processed_dir,region,f"{tb}.json")) as f:
            table_info = json.loads(f.read())
        table_infos_dict[tb] = [
            {
                "table_info": f"Table {tb} has columns: {schema_str[:-2]}",
                "table_summary": f'{table_info["Summary"]}. {table_info["table_summary"]}',
            }
        ]
    return table_infos_dict

In [7]:
us_tb_dict = get_table_infos(us_engine,"US")
india_tb_dict = get_table_infos(india_engine,"India")
china_tb_dict = get_table_infos(china_engine,"China")
europe_tb_dict = get_table_infos(europe_engine,"Europe")
global_tb_dict = get_table_infos(global_engine,"Global")
aus_nz_canada_tb_dict = get_table_infos(aus_nz_canada_engine,"AUS_NZ_CANADA")
japan_tb_dict = get_table_infos(japan_engine,"Japan")
emerging_tb_dict = get_table_infos(emerging_engine,"Emerging")

In [10]:
us_tb_dict

{'DollarUS': [{'table_info': 'Table DollarUS has columns: Industry_Name (VARCHAR), Number_of_firms (INTEGER), Average_Company_Age_years_ (INTEGER), Market_Cap_millions_ (INTEGER), Book_Equity_millions_ (INTEGER), Enteprise_Value_millions_ (INTEGER), Invested_Capital_millions_ (INTEGER), Total_Debt_including_leases_millions_ (INTEGER), Revenues_millions_ (INTEGER), Gross_Profit_millions_ (INTEGER), EBITDA_millions_ (INTEGER), EBIT_Operating_Income_millions_ (INTEGER), Net_Income_millions_ (INTEGER)',
   'table_summary': 'To report aggregated dollar value of key operating and marker numbers, by industry group, in millions of US $.. This table contains financial information about various industries such as their names, number of firms, average company age, market cap, book equity, enterprise value, invested capital, total debt, revenues, gross profit, EBITDA, EBIT, and net income. The data is delimited by a newline character and can be used for comparison and analysis of the financial per

## EMBEDDINGS

1. Embed the table summary and table SCHEMA to get the table that the user is looking for
2. Embed the table rows for each table, so as to get relevant rows from the retrieved table 

## EMBED THE TABLE SUMMARY AND TABLE SCHEMA

In [13]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
from chromadb.utils.batch_utils import create_batches

load_dotenv(override=True)
emb_fn = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ['OPENAI_API_KEY'],
                model_name="text-embedding-3-small")
# EMBEDDING_MODEL = "mixedbread-ai/mxbai-embed-large-v1"
# emb_fn = embedding_functions.HuggingFaceEmbeddingFunction(model_name=EMBEDDING_MODEL,api_key=os.environ["HF_API_KEY"])
def embed_table_info(region:str,tb_dict):
    client = chromadb.PersistentClient(path=f"{region}_TABLE")

    table_collection = client.get_or_create_collection(name="table",embedding_function=emb_fn)

    table_docs = []
    table_metadata = []


    for table_name,table_data in tb_dict.items():
        table_docs.append(table_data[0]['table_info'] + ". " + table_data[0]['table_summary'])
        table_metadata.append({"table_name":table_name,'table_metadata':table_data[0]['table_info']})
    table_ids = [f"id{i}" for i in range(len(table_docs))]
    assert len(table_docs) == len(table_metadata)

    batches = create_batches(api=client,ids=table_ids, documents=table_docs, metadatas=table_metadata)
    for batch in tqdm(batches,desc="Embedding table info"):
        table_collection.add(ids=batch[0],
                    documents=batch[3],
                    metadatas=batch[2])

In [8]:
region = "US"

def embed_rows(region:str,batch_size:int=24):
    client = chromadb.PersistentClient(path=f"{region}_TABLE")

    rows_collection = client.get_or_create_collection(name="rows",embedding_function=emb_fn)

    rows_docs = []
    rows_metadata = []
    region_path = os.path.join(processed_dir,region)
    for df_path in os.listdir(region_path):
        df_full_path = os.path.join(region_path,df_path)
        df = pd.read_csv(df_full_path,index_col=False)
        for idx,row in df.iterrows():
            row_str = ""
            for rv in row.values:
                row_str+=str(rv) + ", "
                row_str = row_str.replace('"',"")
                row_str = row_str.replace("'",'"')
            rows_docs.append(row_str[:-2])
            rows_metadata.append({"table_name":df_path.split(".")[0],"region":region,"index":idx})
    row_ids = [f"id{i}" for i in range(len(rows_docs))]
    # print(len(rows_docs),len(rows_metadata))
    assert len(rows_docs) == len(rows_metadata) == len(row_ids)

    for start in tqdm(range(0,len(rows_docs),batch_size),desc="Embedding rows"):
        end = min(start+batch_size,len(rows_docs))
        batch_ids = row_ids[start:end]
        batch_rows = rows_docs[start:end]
        batch_metadatas = rows_metadata[start:end]
        rows_collection.add(ids=batch_ids,
                    documents=batch_rows,
                    metadatas=batch_metadatas)

In [9]:
# region = "US"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,500)

Embedding rows: 100%|██████████| 5/5 [01:09<00:00, 13.91s/it]


In [10]:
# region = "India"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1000)

Embedding rows: 100%|██████████| 3/3 [00:50<00:00, 16.77s/it]


In [11]:
# region = "China"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1000)

Embedding rows: 100%|██████████| 3/3 [00:49<00:00, 16.60s/it]


In [12]:
# region = "Europe"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1000)

Embedding rows: 100%|██████████| 3/3 [01:09<00:00, 23.28s/it]


In [13]:
# region = "Global"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1000)

Embedding rows: 100%|██████████| 3/3 [00:49<00:00, 16.37s/it]


In [14]:
# region = "Emerging"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1000)

Embedding rows: 100%|██████████| 3/3 [01:13<00:00, 24.62s/it]


In [15]:
# region = "Japan"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1000)

Embedding rows: 100%|██████████| 3/3 [01:09<00:00, 23.07s/it]


In [16]:
# region = "AUS_NZ_CANADA"
# embed_table_info(region,us_tb_dict)
# embed_rows(region,1104)

Embedding rows: 100%|██████████| 3/3 [00:48<00:00, 16.07s/it]


## TEXT-TO-SQL PIPELINE

### LOAD DATABASE

In [11]:
region = "US"

db_dict = {
    "US":us_engine,
    "India":india_engine,
    "China":china_engine,
    "Europe":europe_engine,
    "Global":global_engine,
    "AUS_NZ_CANADA":aus_nz_canada_engine,
    "Japan":japan_engine,
    "Emerging":emerging_engine,
}

def get_collections_db(region:str):
    client = chromadb.PersistentClient(path=f"{region}_TABLE")
    table_collection = client.get_collection(name="table",embedding_function=emb_fn)
    row_collection = client.get_collection(name="rows",embedding_function=emb_fn)
    return [db_dict[region],table_collection,row_collection]

In [14]:
db_collection_dict = {
    "US":get_collections_db("US"),
    "India":get_collections_db("India"),
    "China":get_collections_db("China"),
    "Europe":get_collections_db("Europe"),
    "Global":get_collections_db("Global"),
    "AUS_NZ_CANADA":get_collections_db("AUS_NZ_CANADA"),
    "Japan":get_collections_db("Japan"),
    "Emerging":get_collections_db("Emerging"),
}

In [15]:
db_collection_dict['US']

[Engine(sqlite:///US.db), Collection(name=table), Collection(name=rows)]

In [308]:
import dspy
from typing import List
load_dotenv(override=True)
text_to_sql = dspy.OpenAI(model='gpt-3.5-turbo', max_tokens=500)
# text_to_sql = dspy.Claude(api_key=os.environ['ANTHROPIC_API_KEY'])
sql_to_answer = dspy.OpenAI(model='gpt-3.5-turbo',max_tokens=1024)

class TextToSQLAnswer(dspy.Signature):
    """Convert natural language text to SQL using suitable schema(s) from multiple schema choices"""

    question:str = dspy.InputField(desc="natural language input which will be converted to SQL")
    relevant_table_schemas_rows:str = dspy.InputField(desc="Multiple possible tables which has table name and corresponding columns, along with relevant rows from the table (values in the same order as columns above)")
    sql:str = dspy.OutputField(desc="Generate syntactically correct sqlite query with correct column names using suitable tables(s) and its rows.\n Don't forget to add distinct.\n Please rename the returned columns into suitable names.\n Don't output anything else other than the sqlite query")
    # schemas_used: List[str] = dspy.OutputField(desc="List of schemas from the provided schemas used to generate the SQL output")
# class SQLToAnswer(dspy.Signature):
class SQLReturnToAnswer(dspy.Signature):
    """Answer the question using the rows from the SQL query"""
    question:str = dspy.InputField()
    sql:str = dspy.InputField(desc="sqlite query that generated the rows")
    # relevant_rows:str = dspy.InputField(desc="relevant rows in form of tuple (values in the same order as the select statement in the sql query)")
    relevant_rows:str = dspy.InputField(desc="relevant rows to answer the question")
    answer:str = dspy.OutputField(desc="answer to the question using relevant rows and the sql query")

class SQLRectifier(dspy.Signature):
    """Correct the SQL query to resolve the error using the proper table names, columns and rows"""  
    input_sql:str = dspy.InputField(desc="sqlite query that needs to be fixed")
    error_str: str = dspy.InputField(desc="error that needs to be resolved")
    relevant_table_schemas_rows:str = dspy.InputField(desc="Multiple possible tables which has table name and corresponding columns, along with relevant rows from the table (values in the same order as columns above)")
    sql:str = dspy.OutputField(desc="corrected sqlite query to resolve the error and remove and any invalid syntax in the query.\n Don't output anything else other than the sqlite query")

dspy.settings.configure(lm=text_to_sql)

def process_sql_str(sql_str:str):
    sql_str = sql_str.replace("```","")
    sql_str = sql_str.replace("sql","")
    sql_str = sql_str.strip()
    return sql_str

class TextToSQLQueryModule(dspy.Module):
    def __init__(self,region:str,use_cot:bool=True,max_retries:int=2):
        super().__init__()
        self.region = region
        db,table_collection,row_collection = db_collection_dict[region]
        self.table_collection = table_collection
        self.use_cot = use_cot
        self.db = db
        self.row_collection = row_collection
        if self.use_cot == True:
            self.sqlAnswer = dspy.ChainOfThought(TextToSQLAnswer)
        else:
            self.sqlAnswer = dspy.Predict(TextToSQLAnswer)
        self.final_output = dspy.Predict(SQLReturnToAnswer)
        self.max_tries = max_retries
        self.sql_rectifier = dspy.ChainOfThought(SQLRectifier)
    def forward(self,question:str):
        question_emb = emb_fn.embed_with_retries(question)
        docs = self.table_collection.query(
            # query_texts=question,
            query_embeddings = question_emb,
            n_results = 5
        )
        # relevant_table_schemas = ""
        relevant_rows_schemas = ""
        existing_table_names = []
        for table_idx,metadata_name in enumerate(docs['metadatas'][0]):
            table_metadata = metadata_name['table_metadata']
            # relevant_table_schemas += f'Schema {idx+1}: {table_metadata}\n'
            table_name = metadata_name['table_name']
            if table_name in existing_table_names: 
                continue
            existing_table_names.append(table_name)
            # print(table_name)
            rows = self.row_collection.query(
                # query_texts=question,
                query_embeddings = question_emb,
                n_results = 5,
                where = {"table_name":table_name}
            )
            relevant_rows_schemas += f'Schema {table_idx+1}: {table_metadata}\n'
            for row_idx,row in enumerate(rows['documents'][0]):
                relevant_rows_schemas += f'\tRow {row_idx+1} from table {table_name}: {row}\n'
            relevant_rows_schemas += '\n\n'
        print(relevant_rows_schemas)
        sql_query = self.sqlAnswer(question=question,relevant_table_schemas_rows=relevant_rows_schemas)
        # return sql_query
        num_tries = 0
        print(sql_query)
        while num_tries <= self.max_tries:
            with us_engine.connect() as conn:
                try:
                    result = conn.execute(text(process_sql_str(sql_query.sql)))
                    # print("Results: ",result.fetchall())
                    # results = result.fetchall()
                    # if results == []:
                    #     sql_query = self.sql_rectifier(input_sql=sql_query.sql,error_str="No rows returned. Let's check the table names and row names step by step again and correct the sql query",relevant_table_schemas_rows=relevant_rows_schemas)
                    #     num_tries = self.max_tries
                    num_tries = self.max_tries + 1
                except Exception as error:
                    # return sql_query,error
                    sql_query = self.sql_rectifier(input_sql=sql_query.sql,error_str=str(error),relevant_table_schemas_rows=relevant_rows_schemas)
                    num_tries += 1
                    print(sql_query)
                    if num_tries == self.max_tries:
                        return sql_query,error

        with dspy.context(lm=sql_to_answer):
            row_str = ""
            key = tuple(result.keys())
            for row in result.fetchall():
                for r,k in zip(row,key):
                    row_str += f" {k} = {r},"
                row_str = row_str[:-1]
                row_str += "\n"
            print(row_str)
            final_answer = self.final_output(question=question,sql=sql_query.sql,relevant_rows=row_str)
            return final_answer

tsql = TextToSQLQueryModule("US")
sq = tsql(question="What is the effective tax rate of software industry?")
# sq = tsql(question="What is the beta value and number of firms for all the Software industries and semiconductor industry?")
# sq = tsql("What is the debt to EBITDA ratio for software industry?")

Schema 1: Table wcdata has columns: Industry_Name (VARCHAR), Number_of_firms (INTEGER), Acc_Rec_Sales (INTEGER), Inventory_Sales (INTEGER), Acc_Pay_Sales (INTEGER), Non_cash_WC_Sales (INTEGER)
	Row 1 from table wcdata: Environmental & Waste Services, 57, 0.1610239529643997, 0.0160835818498789, 0.0732441943895783, 0.1018059159059052
	Row 2 from table wcdata: Utility (Water), 13, 0.1519332278688117, 0.0326291583073621, 0.0835453907415389, 0.1135057399930943
	Row 3 from table wcdata: Software (Internet), 35, 0.1228885101454502, 1.6367264776917992e-05, 0.0423722824532171, 0.0966957543939344
	Row 4 from table wcdata: Packaging & Container, 22, 0.1374846253483106, 0.1265656973833159, 0.1305482832092476, 0.1112268155694879
	Row 5 from table wcdata: Semiconductor, 63, 0.1290909417966836, 0.1898317691334856, 0.0849115283669021, 0.2240302287823765


Schema 2: Table wacc has columns: Industry_Name (VARCHAR), Number_of_Firms (INTEGER), Beta (INTEGER), Cost_of_Equity (INTEGER), E_D_E_ (INTEGER), St

In [309]:
sq

Prediction(
    answer='The effective tax rate of the software industry is 0.0260557605345441.'
)

In [310]:
print(sq.answer)

The effective tax rate of the software industry is 0.0260557605345441.
